In [20]:
from aicspylibczi import CziFile
import numpy as np
from numpy import random as rd
import tifffile as tf
from pathlib import Path

from matplotlib import pyplot as plt

import re
from pathlib import Path
import shutil
from os import remove, rmdir
from sys import argv

In [21]:
# helper functions

def isempty(dir_path):
    # Checks a directory to see if it contains any files
    
    path = Path(dir_path)
    has_next = next(path.iterdir(), None)
    if has_next is None:
        return True
    return False

def create_dir(dir_path):
    # Create an empty directory at dir_path if it doesn't yet exist
    try:
        Path.mkdir(dir_path)
    except FileExistsError:
        if not isempty(input_dir):
            raise Exception('Directory ' +dir_path.name + ' is not empty.')

In [22]:
rootdir = Path("/media/austin/IPI_8plex_project/8plex/data/")
savedir = Path("/media/austin/IPI_8plex_project/8plex/analysis/00_tiled/")
files = rootdir.glob("*.czi")

In [23]:
#channels = ['DAPI','FoxP3','CD4','CD45','CD8']

#channels=['Brightfield']

is_bf = False
tile_size = 2048

for f in files:
    
    if Path(savedir,f.stem).exists():
        continue
    
    #f = Path(rootdir, f.name)
    if "CRC" in f.name:
        channels = ['DAPI','HLADR','CD8','CD163','CD4','XCR1','CD3','PDL1','EPCAM']
    elif "MEL" in f.name:
        channels = ['DAPI','HLADR','CD8','CD163','CD4','XCR1','CD3','PDL1','MelanA']
    else:
        channels = ['DAPI','HLADR','CD8','CD163','CD4','XCR1','CD3','PDL1','PanCK']
        
    czi = CziFile(f)
    im_shape = czi.get_dims_shape()
    
    if not is_bf:
        
        nchannels = im_shape[0]['C'][1]
        assert(len(channels)==nchannels)
    elif is_bf:
        
        nchannels = 1

    expdir = Path(savedir, f.stem)
    try:
        Path.mkdir(expdir)
    except FileExistsError: pass

    print("Reading " + f.name)

    for c in np.arange(nchannels):
        
        fov = 0
        print("Reading " + channels[c] + " channel")    
        im = czi.read_mosaic(C=c)
        
        if is_bf:
            _,w,h,rgb = im.shape
        else:
            _,w,h = im.shape

        rows = list(np.arange(0,w,tile_size))
        cols = list(np.arange(0,h,tile_size))
        

        for x in rows:

            if w-x < tile_size:
                x_end = w
            else:
                x_end = x+tile_size

            for y in cols:
                
                
                # create empty tile, useful for padding out incomplete tiles at the edges with zeros

                if is_bf:
                    tile = np.zeros((tile_size, tile_size, 3))
                else:
                    tile = np.zeros((tile_size, tile_size))

                if h-y < tile_size:
                    y_end = h
                else:
                    y_end = y+tile_size
                    
                if is_bf:
                    tile[0:x_end-x, 0:y_end-y,:] = im[0, x:x_end, y:y_end,:]
                else:
                    tile[0:x_end-x, 0:y_end-y] = im[0, x:x_end, y:y_end]


                fovdir = Path(expdir, 'fov' + str(fov))

                try:
                    Path.mkdir(fovdir)
                except FileExistsError:
                    pass
                
                tf.imsave(str(Path(fovdir, channels[c] + ".tiff")), tile)

                if c==0:
                    with open(Path(expdir, "tile_metadata.txt"), "a") as f:
                        f.write(",".join(["fov" + str(fov), str(x), str(x_end), str(y), str(y_end)]) + "\n")

                fov += 1

    fov_dirs = [fov for fov in Path(expdir).iterdir() if fov.is_dir()]

    #Create required directory structure for DeepCell
    input_dir = Path(expdir,"input_data")
    create_dir(input_dir)
    single_tiff_dir = Path(input_dir, "single_channel_inputs")
    create_dir(single_tiff_dir)
    mibitiff_dir = Path(input_dir, "mibitiff_inputs")
    create_dir(mibitiff_dir)
    deepcell_input_dir = Path(input_dir,"deepcell_input")
    create_dir(deepcell_input_dir)
    deepcell_output_dir = Path(expdir, "deepcell_output")
    create_dir(deepcell_output_dir)

    for f in fov_dirs:

        res = re.match("fov\d+",f.name)[0]
        create_dir(Path(single_tiff_dir,res))

        tiff_dir = Path(single_tiff_dir,res,"TIFs")
        
        create_dir(tiff_dir)

        tiffs = [im_file for im_file in Path(f).iterdir() if (im_file.name.endswith(".tif") or im_file.name.endswith(".tiff"))]

        for tif in tiffs:
            shutil.move(str(tif), str(Path(tiff_dir, tif.name)))

        rmdir(f)

Reading IPICRC105T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
Reading IPICRC106T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
Reading IPICRC107T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
Reading IPICRC111T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
Reading IPICRC112T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading C

Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading PanCK channel
Reading IPICRC048T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
Reading IPICRC053T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
Reading IPICRC055T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel
Reading PDL1 channel
Reading EPCAM channel
Reading IPICRC057T1_8plex.czi
Reading DAPI channel
Reading HLADR channel
Reading CD8 channel
Reading CD163 channel
Reading CD4 channel
Reading XCR1 channel
Reading CD3 channel

AssertionError: 